# **GCD_v1 algorithm in with Vlad's lib**
[The detailed algorithm](https://web.eecs.umich.edu/~weimerw/2015-4610/scottcd/10a_thfnd.pdf)
[Helped Wiki](https://en.wikipedia.org/wiki/Lambda_calculus)

In [1]:
from lambda_calculus import Variable as Var
from lambda_calculus import Abstraction as Lambda
from lambda_calculus import Application as App

from lambda_calculus.visitors.normalisation import BetaNormalisingVisitor

In [2]:
from lambda_calculus.terms import logic
from lambda_calculus.terms import arithmetic
from lambda_calculus.terms import pairs
from lambda_calculus.terms import combinators

In [3]:
def multi_app_term(term_0, term_1, *terms):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term

In [4]:
def leq_term():
    m, n = Var("m"), Var("n")
    return Lambda("m", Lambda("n", App(
        arithmetic.ISZERO,
        App(App(arithmetic.SUBTRACT, m), n)
    )))

In [5]:
def eq_term():
    mm, nn = Var("mm"), Var("nn")
    return Lambda("mm", Lambda("nn", multi_app_term(
        logic.AND,
        multi_app_term(leq_term(), mm, nn),
        multi_app_term(leq_term(), nn, mm)
    )))

gcd ≡ (Y
        (λg.λa.λb.(ite (equal a b)
            a
            (ite (greater_than a b)
                (g (minus a b) b)
                (g (minus b a) a)
            )
        )
      )

In [6]:
def gcd_v2():
    g, a, b = Var("g"), Var("a"), Var("b")
    equal_ab = multi_app_term(eq_term(), a, b)

    minus_ab = multi_app_term(g, multi_app_term(arithmetic.SUBTRACT, a, b), b)
    minus_ba = multi_app_term(g, multi_app_term(arithmetic.SUBTRACT, b, a), a)
    le_ab = multi_app_term(leq_term(), b, a)
    inner_ite_term = multi_app_term(logic.IF_THEN_ELSE, le_ab, minus_ab, minus_ba)

    ite_main_term = multi_app_term(logic.IF_THEN_ELSE, equal_ab, a, inner_ite_term)
    gcd_lambda = Lambda("g", Lambda("a", Lambda("b", ite_main_term)))
    return App(combinators.Y, gcd_lambda)

In [7]:
def test_gcd(var_0=0, var_1=0):
    term_ = multi_app_term(gcd_v2(), arithmetic.number(var_0), arithmetic.number(var_1))

    print(BetaNormalisingVisitor().skip_intermediate(term_))

In [14]:
test_gcd(0, 0)

(λf.(λx.x))


In [15]:
test_gcd(1, 0)


KeyboardInterrupt



In [16]:
test_gcd(0, 1)

KeyboardInterrupt: 

In [17]:
test_gcd(2, 2)

(λf.(λx.(f (f x))))


In [18]:
test_gcd(2, 4)

(λf.(λx.(f (f x))))


In [19]:
test_gcd(4, 2)

(λf.(λx.(f (f x))))


In [20]:
test_gcd(6, 3)

(λf.(λx.(f (f (f x)))))


In [21]:
test_gcd(3, 6)

(λf.(λx.(f (f (f x)))))


In [22]:
test_gcd(16, 32)

(λf.(λx.(f (f (f (f (f (f (f (f (f (f (f (f (f (f (f (f x))))))))))))))))))


In [23]:
test_gcd(15, 0)

KeyboardInterrupt: 

In [24]:
test_gcd(15, 1)

(λf.(λx.(f x)))


In [25]:
test_gcd(1, 15)

(λf.(λx.(f x)))


In [26]:
test_gcd(12, 15)

(λf.(λx.(f (f (f x)))))


In [8]:
test_gcd(144, 12)

KeyboardInterrupt: 

In [10]:
def gcd_v1_1():
    g, a, b = Var("g"), Var("a"), Var("b")
    equal_ab = multi_app_term(eq_term(), a, b)

    minus_ab = multi_app_term(g, multi_app_term(arithmetic.SUBTRACT, a, b), b)
    minus_ba = multi_app_term(g, multi_app_term(arithmetic.SUBTRACT, b, a), a)
    le_ab = multi_app_term(leq_term(), b, a)
    inner_ite_term = multi_app_term(logic.IF_THEN_ELSE, le_ab, minus_ab, minus_ba)

    ite_left_term = multi_app_term(logic.IF_THEN_ELSE, equal_ab, a, inner_ite_term)
    main_or_term = multi_app_term(logic.OR, App(arithmetic.ISZERO, a), App(arithmetic.ISZERO, b))
    ite_main_term = multi_app_term(logic.IF_THEN_ELSE, main_or_term, arithmetic.number(1), ite_left_term)
    gcd_lambda = Lambda("g", Lambda("a", Lambda("b", ite_main_term)))
    return App(combinators.Y, gcd_lambda)

def test_gcd_v1_1(var_0=0, var_1=0):
    term_ = multi_app_term(gcd_v1_1(), arithmetic.number(var_0), arithmetic.number(var_1))

    print(BetaNormalisingVisitor().skip_intermediate(term_))

In [11]:
test_gcd_v1_1(0, 0)

(λf.(λx.(f x)))


In [12]:
test_gcd_v1_1(1, 0)

(λf.(λx.(f x)))


In [13]:
test_gcd_v1_1(0, 1)

(λf.(λx.(f x)))


In [14]:
test_gcd_v1_1(2, 4)

(λf.(λx.(f (f x))))


In [15]:
test_gcd_v1_1(4, 2)

(λf.(λx.(f (f x))))


In [16]:
test_gcd_v1_1(5, 10)

(λf.(λx.(f (f (f (f (f x)))))))


In [17]:
test_gcd_v1_1(10, 5)

(λf.(λx.(f (f (f (f (f x)))))))


In [18]:
test_gcd_v1_1(20, 3)

(λf.(λx.(f x)))


In [19]:
test_gcd_v1_1(3, 20)

(λf.(λx.(f x)))
